In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
import time

sys.path.insert(0, "/home/ylu/project/Benchmark_alignment_methods/scripts/methods")
from my_STAlign import stalign_align
sys.path.insert(0, "/home/ylu/project")
from utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data_folder = "../../data/STARMapPlus/"
method = 'STAlign'
i = 3
adata = ad.read_h5ad(os.path.join(data_folder, f"sagittal{i}.h5ad"))
scale = np.linalg.norm(adata.obsm['spatial'].max(0) - adata.obsm['spatial'].min(0))
slices = split_slice(
    adata=adata,
    spatial_key='spatial_raw',
    split_num=2,
    axis=2,
)

In [5]:
overlay_ratio_set = np.arange(0.5, 1.05, 0.05)
from tqdm import tqdm
maes = []
for overlay_ratio in tqdm(overlay_ratio_set):
    slice1 = slices[0].copy()
    slice2 = slices[1].copy()
    slice1_crop, slice2_crop = crop_slices(
        slice1, 
        slice2, 
        spatial_key='spatial', 
        overlay_ratio = overlay_ratio,
    )
    slice1, slice2 = slice1_crop.copy(), slice2_crop.copy()
    # sampling_num = 20000
    # sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    # sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    # slice1 = slice1[sampline_idx1,:]
    # slice2 = slice2[sampline_idx2,:]
    spatial_key = 'spatial'
    key_added = 'align_spatial'
    
    align_slices = stalign_align(
        models = [slice1, slice2],
        spatial_key = spatial_key,
        key_added = key_added,
        device = "cuda:0",
        scale=0.13,
        zero_mean=False,
    )
    spatial_align = align_slices[1].obsm[key_added]
    spatial_GT = align_slices[1].obsm['spatial']
    mae, mae_var = MAE(spatial_align, spatial_GT) / scale
    maes.append(mae)
np.save(f'./results/{method}_different_overlap.npy', {'maes': maes, 'overlay_ratio':overlay_ratio}, allow_pickle=True)

  0%|                                      | 0/11 [00:00<?, ?it/s]/home/ylu/project/STalign/STalign/STalign.py:1044: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L = torch.tensor(L,device=device,dtype=dtype,requires_grad=True)
/home/ylu/project/STalign/STalign/STalign.py:1045: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  T = torch.tensor(T,device=device,dtype=dtype,requires_grad=True)
/home/ylu/anaconda3/envs/STAlign/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # 